---
title: "Fallstudie für Lidl: Erweiterung des Süßwarensortiments (Eigenmarke)"
format:
  html:
    code-fold: true
jupyter: python3
---


### Datenexploration

#### Daten:

- 9 Characteristika von bereits existierenden Süßigkeiten. (boolean)
- sugarpercent: Der Perzentilwert des Zuckergehalts innerhalb des Datensatzes.
- pricepercent: Der Stückpreis-Prozentwert im Vergleich zum Rest der Menge.
- winpercent: Der Gesamtsiegprozentsatz basierend auf 269.000 Matchups.

#### Ziel:

- Analysiere die Auswirkungen der Charakteristika von Süßwaren auf deren Beliebtheit
- Gebe eine Empfehlung auf Basis dieser Analyse für die Eigenschaften einer neuen Süßigkeit.


In [ ]:
#| fig-cap: "Figure: Best and worst performing candy"

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm


df_original = pd.read_csv('candy-data.csv')
#pd.set_option("display.max_rows", None)
df = df_original.copy()
df['winpercent'] = df_original['winpercent']/100
del(df_original)

df_sorted = df.sort_values(by='winpercent', ascending=False)

df_filtered = df_sorted.drop(
    columns=[
        'caramel', 
        'peanutyalmondy', 
        'nougat', 
        'crispedricewafer', 
        'hard', 
        'bar', 
        'pluribus'
    ], 
    inplace=False
)
df_filtered

#### Probleme mit dieser Analyse
- Beliebtheit ist etwas biased, da der Preis in dem Ranking keine Rolle spielt.
  - Marketing, Brand reputation, packaging, current and future market trends sind auch nicht berücksichtigt.
  - Preis beinhaltet hier auch höheren Wert durch größere Einzelmenge
- Ebenso scheint der Zuckergehalt nicht prozentual zu sein, sondern bezieht sich auf absolute Werte.
  - Zuckerranking bedeuted hier oft nicht süßer, sondern einfach dass das Produktgewicht höher ist.

#### Was macht eine gute Empfehlung aus?
- Sollte möglichst belieble Eigenschaften haben
- Sollte nicht zu ähnlich zu Produkten bereits auf dem Markt sein --> we need some uniqueness to stand out
- Manche Eigenschaften sind nicht (gut) kombinierbar
- Manche Eigenschaften sind (nur) in Kombination begehrt
  - Interactions müssen hier analysiert werden!


In [ ]:
print('Do we have any NaN values:')
df.isnull().values.any()

In [ ]:
#| fig-cap: "Figure: Summary Statistics for OLS:  winpercent ~ pricequartile"
sns.regplot(x='pricepercent', y='winpercent', data=df)
X = sm.add_constant(df['pricepercent'])
sm.OLS(df.winpercent,X).fit().summary()

#### Einfluss der Preis Variable

- Es besteht eine klare pos. Korrelation zw. Preis und Beliebtheit.
- fruity, hard, pluribus:
  - einzigen vars die im Schnitt Süßigkeiten unbeliebler machen
  - auch die einzigen vars die Süßigkeiten billiger machen
- Billigsegment verliert gegen teurere Bars im direkten Vergleich, aber unklar ob das auch im Supermarkt mit Preisen der Fall ist
  - Könnten im Supermarkt besser performen
  - Könnten ein Nichenprodukt sein
  - Preis ist hier pro Stück, größere Riegel könnten kleine Packungen mit Bonbons mit purer Masse schlagen

#### Correlation matrix


In [ ]:
#| fig-cap: "Best and worst performing candy"
corr = df.iloc[:, 1:].corr() #correlation matrix
mask = np.triu(np.ones_like(corr, dtype=bool)) # mask for upper triangle
f, ax = plt.subplots(figsize=(8, 8))
cmap = sns.diverging_palette(230, 20, as_cmap=True) # Colormap
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Hier sehen wir welche Eigenschaften öfters zusammen (rot) oder exkludierend (blau) befunden werden:

- fruity, hard, fruity oft zusammen
- rest oft zusammen
- fruity, hard, pluribus selten mit rest

--> Sehen wir hier zwei Hauptgruppen: Schokolade und fruity/hard candy?

#### Clusteranalysis

- Können wir Marktsegmente identifizieren?


In [ ]:
# Preprocessing
from sklearn.preprocessing import StandardScaler
df_YX = df.select_dtypes(include=[float, int])
scaler = StandardScaler()
df_normalized = scaler.fit_transform(df_YX)

# Distance Matrix
from scipy.spatial.distance import pdist, squareform
distance_matrix = pdist(df_normalized, metric='euclidean')
# distance_matrix = squareform(distance_matrix)

# Hierarchical Clustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt

# Perform hierarchical clustering
linked = linkage(distance_matrix, method='ward')

# Plot the dendrogram
plt.figure(figsize=(8, 8))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
plt.title('Dendrogram')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

# Determine Clusters
num_clusters = 2 # Specify the number of clusters
clusters = fcluster(linked, num_clusters, criterion='maxclust')

# Add cluster labels to the original DataFrame
df_YX['Cluster'] = clusters

# Visualize clusters
plt.figure(figsize=(8, 8))
sns.scatterplot(x=df_YX['pricepercent'], y=df_YX['winpercent'], hue=df_YX['Cluster'], palette='viridis')
plt.title('Clusters in Feature Space')
plt.xlabel('Price percentile')
plt.ylabel('Winning Percentage')
plt.legend(title='Cluster')
plt.show()

cluster_means = df_YX.groupby('Cluster').mean()
print(cluster_means)

#### Clearly we have two different segments

Should we handle them differently?

- Compare how different they are
  - Visualize data on the first two principal components w.r.t. their attributes


In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px

# Select and copy relevant columns from the original DataFrame
df_attr = df_YX[['chocolate', 'fruity', 'caramel', 'peanutyalmondy', 
                 'nougat', 'crispedricewafer', 'hard', 'bar', 'pluribus']].copy()

# Perform PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df_attr)

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(df_attr)

# Add PCA and t-SNE results to the DataFrame
df_attr['pca-2d-one'] = X_pca[:, 0]
df_attr['pca-2d-two'] = X_pca[:, 1]
df_attr['tsne-2d-one'] = tsne_results[:, 0]
df_attr['tsne-2d-two'] = tsne_results[:, 1]

# Plot t-SNE results for selected columns
# selected_columns_tsne = ['fruity', 'peanutyalmondy', 'crispedricewafer', 'hard', 'pluribus']
# for column in selected_columns_tsne:
#      fig_tsne = px.scatter(df_attr, x='tsne-2d-one', y='tsne-2d-two', 
#                           color=df_attr[column].astype(str),
#                           title=f't-SNE colored by {column}',
#                           labels={column: column})
#     fig_tsne.show()

color_map = { '0': 'blue', '1': 'red' }
# Plot PCA results for all feature columns
selected_columns_pca = ['fruity','hard','nougat','crispedricewafer','peanutyalmondy', 'chocolate','bar','pluribus']
for column in selected_columns_pca:
    fig_pca = px.scatter(df_attr, x='pca-2d-one', y='pca-2d-two', 
                        color=df_attr[column].astype(str),
                        title=f'PCA colored by {column}',
                        color_discrete_map=color_map,
                        labels={column: column})
    fig_pca.show()

#### How many ingredient based attributes do the sweets have? (i.e. we exclude pluribus, bar, and hard)


In [ ]:
df_attr = df_YX[['chocolate', 'fruity', 'caramel', 'peanutyalmondy', 
                 'nougat', 'crispedricewafer']].copy()
sums = df_attr.sum(axis=1)
sum_counts = sums.value_counts().sort_index()
sum_counts_df = pd.DataFrame({
    'Sum': sum_counts.index,
    'Count': sum_counts.values
})
fig = px.bar(sum_counts_df, x='Sum', y='Count',
             title='Sum of Each Feature Column',
             labels={'Feature': 'Feature', 'Sum': 'Sum'},
             color='Sum',
             color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
data = pd.DataFrame({
    'Number of Attributes': sums,  # Replace `sums` with your independent variable
    'Popularity': df['winpercent'] # Replace `y` with your dependent variable (popularity score)
})

# Calculate the mean popularity for each number of attributes
mean_popularity = data.groupby('Number of Attributes')['Popularity'].mean().reset_index()

# Create the scatter plot with a Gaussian kernel regression line
plt.figure(figsize=(10, 6))
sns.lmplot(x='Number of Attributes', y='Popularity', data=data, 
           scatter_kws={'s': 50},  # size of scatter points
           line_kws={'color': 'red'},  # color of the regression line
           lowess=True,  # Gaussian kernel smoothing (locally weighted regression)
           )

# Plot mean popularity points
plt.scatter(mean_popularity['Number of Attributes'], mean_popularity['Popularity'],
            color='blue', label='Mean Popularity', zorder=5)

# Annotate mean popularity points
for i in range(len(mean_popularity)):
    plt.annotate(f"{mean_popularity['Popularity'].iloc[i]:.2f}",
                 (mean_popularity['Number of Attributes'].iloc[i], 
                  mean_popularity['Popularity'].iloc[i]),
                 textcoords="offset points", xytext=(0,5), ha='center')

# Customize the plot
plt.title('Popularity vs. Number of Attributes with Gaussian Kernel Regression')
plt.xlabel('Number of Attributes')
plt.ylabel('Popularity')
plt.legend()
plt.show()